In [44]:
import torch
from torch._export import capture_pre_autograd_graph

from torchvision.models import mobilenet_v2
# from torchvision.models.quantization import mobilenet_v2
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import os
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import copy

In [45]:
def evaluate_model(model, data_loader,device):

    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test images: {accuracy}%')
    return accuracy


In [46]:
def prepare_data(batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to match MobileNet input size
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False,drop_last=True)
    return train_loader, test_loader

batch_size = 16
train_loader, test_loader = prepare_data(batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [47]:
def calibrate(model, data_loader):
    # model.eval()
    torch.ao.quantization.move_exported_model_to_train(model)
    with torch.no_grad():
        for images, labels in data_loader:
            model(images)

In [48]:

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(f'device: {device}')
device = torch.device('cpu')
weight_path = '/home/aa35037123/Wesley/edge_ai/lab3/mobilenetv2_0.963.pth'
model = torch.load(weight_path, map_location=torch.device('cpu'))
model.to('cpu')
print(next(model.parameters()).is_cuda)
model.eval()
example_inputs = (torch.randn(1, 3, 224, 224),)
exported_model = capture_pre_autograd_graph(model, example_inputs)
# Step 2. quantization
from torch.ao.quantization.quantize_pt2e import (
  prepare_pt2e,
  convert_pt2e,
)
from torch.ao.quantization.quantizer.xnnpack_quantizer import (
  XNNPACKQuantizer,
  get_symmetric_quantization_config,
)
# from torch.ao.quantization.quantizer import (
#   XNNPACKQuantizer,
#   get_symmetric_quantization_config,
# )
# backend developer will write their own Quantizer and expose methods to allow
# users to express how they
# want the model to be quantized
quantizer = XNNPACKQuantizer().set_global(get_symmetric_quantization_config())
prepared_model = prepare_pt2e(exported_model, quantizer)
# print(prepared_model.graph)
# calibration omitted
calibrate(prepared_model, test_loader)  # run calibration on sample data
ptq_quantized_model = convert_pt2e(prepared_model)
print(ptq_quantized_model)
# we have a model with aten ops doing integer computations when possible

False
GraphModule()



def forward(self, x):
    arg0, = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
    arg0_1 = arg0
    quantize_per_tensor_default = torch.ops.quantized_decomposed.quantize_per_tensor.default(arg0_1, 1.0, 0, -128, 127, torch.int8);  arg0_1 = None
    dequantize_per_tensor_default = torch.ops.quantized_decomposed.dequantize_per_tensor.default(quantize_per_tensor_default, 1.0, 0, -128, 127, torch.int8);  quantize_per_tensor_default = None
    _frozen_param0 = self._frozen_param0
    dequantize_per_tensor_default_1 = torch.ops.quantized_decomposed.dequantize_per_tensor.default(_frozen_param0, 1.0, 0, -127, 127, torch.int8);  _frozen_param0 = None
    features_0_0_weight_bias = self.features_0_0_weight_bias
    conv2d = torch.ops.aten.conv2d.default(dequantize_per_tensor_default, dequantize_per_tensor_default_1, features_0_0_weight_bias, [2, 2], [1, 1]);  dequantize_per_tensor_default = dequantize_per_tensor_default_1 = features_0_0_weight_bias = None
    quan

/home/aa35037123/miniconda3/envs/lab3/lib/python3.10/site-packages/torch/ao/quantization/observer.py:1272: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
/home/aa35037123/miniconda3/envs/lab3/lib/python3.10/site-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [49]:
# Export the model and Save ExportedProgram
pt2e_quantized_model_file_path =  "./mobilenet_quantized.pt"
# capture the model to get an ExportedProgram
example_inputs = (torch.randn(1, 3, 224, 224),)
quantized_ep = torch.export.export(ptq_quantized_model, example_inputs)
# use torch.export.save to save an ExportedProgram
torch.export.save(quantized_ep, pt2e_quantized_model_file_path)
print('Modle has been export!')

Modle has been export!


In [50]:
loaded_quantized_ep = torch.export.load(pt2e_quantized_model_file_path)
loaded_quantized_model = loaded_quantized_ep.module()
acc = evaluate_model(loaded_quantized_model, test_loader, device)

RuntimeError: Expected input at *args[0].shape[0] to be equal to 1, but got 16